In [5]:
import pandas as pd
import numpy as np
import math



In [ ]:
df=pd.read_csv("C:/Users/HP/OneDrive/Desktop/bopm/project.csv")

In [8]:
df["start"]= ((df["Close"])/(df["Open"])-1)


In [12]:

df.head(10)



,Open,High,Low,Close,Adj Close,Volume,volat
Date,,,,,,,
2022-01-03,17387.150391,17646.650391,17383.300781,17625.699219,17625.699219,200500,NaN
2022-01-04,17681.400391,17827.599609,17593.550781,17805.250000,17805.250000,247400,0.010187
2022-01-05,17820.099609,17944.699219,17748.849609,17925.250000,17925.250000,251500,0.006740
2022-01-06,17768.500000,17797.949219,17655.550781,17745.900391,17745.900391,236500,-0.010005
2022-01-07,17797.599609,17905.000000,17704.550781,17812.699219,17812.699219,239300,0.003764
2022-01-10,17913.300781,18017.449219,17879.150391,18003.300781,18003.300781,232400,0.010700
2022-01-11,17997.750000,18081.250000,17964.400391,18055.750000,18055.750000,220200,0.002913
2022-01-12,18170.400391,18227.949219,18128.800781,18212.349609,18212.349609,245000,0.008673
2022-01-13,18257.000000,18272.250000,18163.800781,18257.800781,18257.800781,303700,0.002496


In [10]:
def volatility(df):
    df['volat']=df['Adj Close'].pct_change()
    vol =df['volat'].std(ddof=0)*np.sqrt(252)
    return vol

In [11]:
volatility (df)


0.17256999226570358

In [6]:
df.head(10)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-03-03 00:00:00+05:30,17451.25,17644.75,17427.699219,17594.349609,356200,0.0,0.0


In [ ]:
def alphaProbability(r,dt,v):
    u=np.exp(v*np.sqrt(dt))
    d=1/u
    rnp=(np.ezp(r*dt)-d)/(u-d)
    return rnp

In [7]:
import yfinance as yf

In [4]:
#latest price of Nifty50
nifty50 =yf.Ticker("^NSEI")
df= nifty50.history(period="1s")
S = df["Close"]
print(S)


Date
2023-03-03 00:00:00+05:30    17594.349609
Name: Close, dtype: float64


In [9]:
#One year Nifty50 data. (y-finance)
df = yf.download("^NSEI", start="2022-01-01", end="2022-12-31")
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-03,17387.150391,17646.650391,17383.300781,17625.699219,17625.699219,200500
2022-01-04,17681.400391,17827.599609,17593.550781,17805.250000,17805.250000,247400
2022-01-05,17820.099609,17944.699219,17748.849609,17925.250000,17925.250000,251500
2022-01-06,17768.500000,17797.949219,17655.550781,17745.900391,17745.900391,236500
2022-01-07,17797.599609,17905.000000,17704.550781,17812.699219,17812.699219,239300
...,...,...,...,...,...,...
2022-12-26,17830.400391,18084.099609,17774.250000,18014.599609,18014.599609,176700
2022-12-27,18089.800781,18149.250000,17967.449219,18132.300781,18132.300781,214300
2022-12-28,18084.750000,18173.099609,18068.349609,18122.500000,18122.500000,193900


In [13]:
# Determine the Option Value in cases of both CALL and PUT.
def binomial_option_pricing(S, K, r, T, volat, N, option_type, show_tree=True):
  dt = T/N
  u = math.exp(volat*math.sqrt(dt))
  d = 1/u
  #Calculation of the risk neutral probability.
  p = (math.exp(r*dt)-d)/(u-d)

  stock_prices = [[S * u**j * d**(i-j) for j in range(i+1)] for i in range(N+1)]
  option_values = [[0 for j in range(i+1)] for i in range(N+1)]

  for j in range(N+1):
        if option_type == 'call':
            option_values[N][j] = max(stock_prices[N][j]-K, 0)
        elif option_type == 'put':
            option_values[N][j] = max(K-stock_prices[N][j], 0)

  for i in range(N-1, -1, -1):
        for j in range(i+1):
            if option_type == 'call':
                option_values[i][j] = max(stock_prices[i][j]-K,math.exp(-r*dt)*(p*option_values[i+1][j]+(1-p)*option_values[i+1][j+1]))
            elif option_type == 'put':
                option_values[i][j] = max(K-stock_prices[i][j], math.exp(-r*dt)*(p*option_values[i+1][j]+(1-p)*option_values[i+1][j+1]))

  if show_tree:
        for i in range(N+1):
            print(' '.join([f'{val:.2f}' for val in option_values[i]]))
  #This function returns the required option values at the very start of the binomial tree.
  return option_values[0][0]

In [14]:
#Let Parameters be
S = 17594.34
K = S
T = 200/365
r = 0.06

In [15]:
#No. of periods(N)=4;
N = 4


In [19]:
#lets define volatility, so it can be easily accessible
df['Log Returns'] = np.log(df['Close']) - np.log(df['Close'].shift(1))
volat = np.sqrt(252) * df['Log Returns'].std()
volat

0.17325337010939032

In [20]:
call_price = binomial_option_pricing(S, K, r, T, volat, N, 'call')
print(f"Call Price: {call_price:.2f}")

737.60
233.88 1362.49
0.00 522.02 2407.52
0.00 0.00 1165.18 3732.13
0.00 0.00 0.00 2407.52 5144.47
Call Price: 737.60


In [21]:
put_price = binomial_option_pricing(S, K, r, T, volat, N, 'put')
print(f"Put Price: {put_price:.2f}")

958.37
1481.81 340.55
2209.00 626.24 0.00
3113.40 1151.61 0.00 0.00
3980.57 2117.74 0.00 0.00 0.00
Put Price: 958.37


In [22]:
#No. of periods(N)=5;
N = 5

In [23]:
call_price = binomial_option_pricing(S, K, r, T, volat, N, 'call')
print(f"Call Price: {call_price:.2f}")

802.10
357.11 1342.72
97.10 671.40 2160.24
0.00 213.94 1224.95 3303.38
0.00 0.00 471.38 2138.53 4536.99
0.00 0.00 0.00 1038.61 3303.38 5843.42
Call Price: 802.10


In [24]:
put_price = binomial_option_pricing(S, K, r, T, volat, N, 'put')
print(f"Put Price: {put_price:.2f}")


1053.03
1517.98 515.47
2118.70 825.71 154.07
2841.39 1290.10 285.54 0.00
3629.20 1945.80 529.18 0.00 0.00
4386.56 2781.20 980.72 0.00 0.00 0.00
Put Price: 1053.03


In [28]:
#No. of periods(N)=10;
N = 10

In [26]:
call_price = binomial_option_pricing(S, K, r, T, volat, N, 'call')
print(f"Call Price: {call_price:.2f}")

802.10
357.11 1342.72
97.10 671.40 2160.24
0.00 213.94 1224.95 3303.38
0.00 0.00 471.38 2138.53 4536.99
0.00 0.00 0.00 1038.61 3303.38 5843.42
Call Price: 802.10


In [29]:
put_price = binomial_option_pricing(S, K, r, T, volat, N, 'put')
print(f"Put Price: {put_price:.2f}")

988.29
1328.21 611.20
1740.20 872.10 320.75
2217.52 1213.18 492.90 128.51
2743.31 1639.07 740.56 216.57 29.94
3292.68 2141.93 1082.48 359.49 56.63 0.00
3840.08 2697.38 1529.46 585.12 107.11 0.00 0.00
4370.50 3267.79 2071.91 927.37 202.59 0.00 0.00 0.00
4884.55 3820.43 2666.39 1414.85 383.17 0.00 0.00 0.00 0.00
5382.81 4355.92 3242.27 2034.52 724.73 0.00 0.00 0.00 0.00 0.00
5865.84 4874.88 3800.20 2634.71 1370.75 0.00 0.00 0.00 0.00 0.00 0.00
Put Price: 988.29


In [30]:
#No. of periods(N)=20;
N = 20

In [31]:
call_price = binomial_option_pricing(S, K, r, T, volat, N, 'call')
print(f"Call Price: {call_price:.2f}")

759.56
527.22 1015.41
346.11 726.43 1333.86
212.05 493.41 982.89 1720.97
119.09 314.11 690.53 1304.93 2180.34
59.78 184.14 456.84 947.60 1698.88 2712.60
25.83 96.98 279.78 651.39 1273.70 2168.13 3303.38
9.05 44.22 154.81 416.95 909.15 1675.41 2712.60 3911.34
2.29 16.44 74.65 242.71 608.30 1240.18 2155.53 3303.38 4536.99
0.32 4.45 29.56 124.04 372.88 866.97 1651.21 2712.60 3911.34 5180.84
0.00 0.66 8.60 52.52 202.43 559.94 1204.60 2143.66 3303.38 4536.99 5843.42
0.00 0.00 1.39 16.49 91.97 323.52 819.53 1628.43 2712.60 3911.34 5180.84 6525.27
0.00 0.00 0.00 2.91 31.34 158.36 504.65 1165.54 2138.53 3303.38 4536.99 5843.42 7226.97
0.00 0.00 0.00 0.00 6.08 58.98 267.21 765.18 1605.92 2712.60 3911.34 5180.84 6525.27 7949.08
0.00 0.00 0.00 0.00 0.00 12.73 109.60 439.91 1122.31 2138.53 3303.38 4536.99 5843.42 7226.97 8692.19
0.00 0.00 0.00 0.00 0.00 0.00 26.66 200.38 702.50 1583.73 2712.60 3911.34 5180.84 6525.27 7949.08 9456.93
0.00 0.00 0.00 0.00 0.00 0.00 0.00 55.80 358.64 1079.70 2138.53 3

In [32]:
put_price = binomial_option_pricing(S, K, r, T, volat, N, 'put')
print(f"Put Price: {put_price:.2f}")

998.65
1242.35 736.46
1521.15 942.76 514.15
1832.57 1186.95 679.85 335.31
2172.07 1468.86 883.78 459.91 200.67
2533.29 1785.84 1128.42 620.18 286.82 107.46
2908.81 2132.72 1413.90 821.15 403.27 160.88 49.60
3291.08 2502.16 1737.40 1066.19 556.90 237.22 78.23 18.58
3673.54 2885.55 2092.89 1355.91 754.08 343.91 121.75 31.06 5.04
4051.38 3274.37 2471.69 1687.23 999.45 489.24 186.69 51.39 9.00 0.75
4421.82 3661.55 2863.63 2053.00 1294.38 681.44 281.43 84.07 15.96 1.44 0.00
4783.85 4042.44 3259.00 2442.56 1635.47 927.07 416.07 135.65 28.13 2.76 0.00 0.00
5137.44 4414.92 3650.38 2843.63 2013.83 1228.72 601.48 215.43 49.17 5.30 0.00 0.00 0.00
5482.89 4778.58 4033.73 3245.10 2415.84 1582.60 847.25 335.68 85.10 10.17 0.00 0.00 0.00 0.00
5820.57 5133.71 4407.96 3639.72 2826.16 1976.92 1158.26 511.19 145.53 19.54 0.00 0.00 0.00 0.00 0.00
6150.89 5480.57 4773.31 4024.92 3232.41 2393.11 1530.11 756.96 245.09 37.52 0.00 0.00 0.00 0.00 0.00 0.00
6474.36 5819.49 5130.02 4400.94 3628.93 2811.34 1945.50